In [1]:
import os
import numpy as np
import pandas as pd
import mne
from mne.preprocessing import ICA
from mne_icalabel import label_components
import seaborn as sns
import matplotlib.pyplot as plt
import xlsxwriter

In [2]:
def compute_psd(raw_data, fmin, fmax, sfreq):
    n_overlap_half = int(2048 / 2)
    psds, freqs = mne.time_frequency.psd_array_welch(
        raw_data, fmin=fmin, fmax=fmax, n_fft=2048, sfreq=sfreq, n_overlap=n_overlap_half
    )
    return psds, freqs

def normalize_psd(psds):
    total_power = np.sum(psds, axis=1, keepdims=True)
    normalized_psds = psds / total_power
    return normalized_psds

def apply_ica(raw, f_min, f_max):
    raw_filter = raw.copy().filter(l_freq=f_min, h_freq=f_max)

    ica = ICA(n_components=5, random_state=0)
    ica.fit(raw_filter)
    
    ic_labels = label_components(raw_filter, ica, method="iclabel")
    exclude_idx = [idx for idx, label in enumerate(ic_labels["labels"]) if label not in ["brain", "muscle artifact"]]

    reconst_raw = raw.copy()
    ica.apply(reconst_raw, exclude=exclude_idx)

    return reconst_raw.get_data() * 1000000

def compute_entropy(psds, low, high):
nd indices corresponding to the frequency band
    freqs = np.linspace(low, high, psds.shape[1])
    
    # Normalize PSD values within the frequency band
    psds_band = psds[:, (freqs >= low) & (freqs <= high)]
    psds_band /= np.sum(psds_band, axis=1, keepdims=True)
    
    # Compute entropy
    entropy_value = -np.sum(psds_band * np.log2(psds_band), axis=1)
    
    return entropy_value


def compute_centroid(psds, freqs, low, high):

    # Find indices corresponding to the frequency band
    indices = np.where((freqs >= low) & (freqs <= high))[0]
    
    # Select PSD values within the frequency band
    psds_band = psds[:, indices]
    
    # Compute centroid
    centroid = np.sum(psds_band * freqs[indices], axis=1) / np.sum(psds_band, axis=1)
    
    return centroid

def compute_centroid_y(psds, freqs, low, high):

    # Find indices corresponding to the frequency band
    indices = np.where((freqs >= low) & (freqs <= high))[0]
    
    # Select PSD values within the frequency band
    psds_band = psds[:, indices]
    
    # Compute centroid
    centroidy = np.sum(psds_band * freqs[indices], axis=1) / np.sum(freqs[indices])
    
    return centroidy

def compute_summed_psd(psds, freqs, low, high):

    # Find indices corresponding to the frequency band
    indices = np.where((freqs >= low) & (freqs <= high))[0]
    
    # Select PSD values within the frequency band
    psds_band = psds[:, indices]
    
    # Compute centroid
    summed_psd = np.sum(psds_band, axis=1)
    
    return summed_psd


In [3]:

# Define a function to calculate the summed PSD for each frequency band
def calculate_summed_psd(psds, freqs, freq_bands):
    summed_psd = {}
    for band, (fmin, fmax) in freq_bands.items():
        freq_indices = (freqs >= fmin) & (freqs <= fmax)
        summed_psd[band] = psds[:, freq_indices].sum(axis=1)
    return summed_psd

# Initialize lists to store data
summed_psd_list = []

In [4]:
data_folders = {
    "Baik": "D:\VSCode\Tesis\Pengukuran\Raw\Base\L3",
    "Cukup": "D:\VSCode\Tesis\Pengukuran\Raw\Base\L2",
    "Kurang": "D:\VSCode\Tesis\Pengukuran\Raw\Base\L1"
}

channel_names = ['AF3', 'T7', 'Pz', 'T8', 'AF4']
freq_bands = {'Delta': (0.5, 4), 'Teta': (4, 8), 'Alfa': (8, 13), 'Beta': (13, 30), 'Gamma': (30, 45)}

labels = []
psd_list = []
normalized_psds_list = []
centroid_list = []

# Initialize lists to store data
summed_psd_data = []

In [5]:
freq_min_band = 2
freq_max_band = 45

In [6]:
dfs = []
for label, folder in data_folders.items():
    eeg_files = [file for file in os.listdir(folder) if file.endswith('.set')]
    
    for eeg_file in eeg_files:
        raw = mne.io.read_raw_eeglab(os.path.join(folder, eeg_file), preload=True)
        sfreq = raw.info['sfreq']
        rawR1 = raw.copy()
        rawT = raw.copy()
        rawR2 = raw.copy()
        raw_rest_1 = rawR1.crop(tmin=0, tmax=180)
        raw_tafakur = rawT.crop(tmin=180, tmax=780)
        raw_rest_2 = rawR2.crop(tmin=780, tmax=945)

        cleaned_data_rest_1 = apply_ica(raw_rest_1, freq_min_band, freq_max_band)
        cleaned_data_tafakur = apply_ica(raw_tafakur, freq_min_band, freq_max_band)
        cleaned_data_rest_2 = apply_ica(raw_rest_2, freq_min_band, freq_max_band)
        
        psdsR1, freqsR1 = compute_psd(cleaned_data_rest_1, freq_min_band, freq_max_band, sfreq)
        psdsT, freqsT = compute_psd(cleaned_data_tafakur, freq_min_band, freq_max_band, sfreq)
        psdsR2, freqsR2 = compute_psd(cleaned_data_rest_2, freq_min_band, freq_max_band, sfreq)

        normalized_psdsR1 = normalize_psd(psdsR1)
        normalized_psdsT = normalize_psd(psdsT)
        normalized_psdsR2 = normalize_psd(psdsR2)
            
        short_label = eeg_file[:2]
            
        n_channels = normalized_psdsR1.shape[0]
        

        # Loop through frequency bands
        for band, (low, high) in freq_bands.items():
            # Calculate centroid and entropy for each frequency band
            
            summed_psdR1 = compute_summed_psd(normalized_psdsR1,freqsR1, low,high)
            summed_psdT = compute_summed_psd(normalized_psdsT,freqsT, low,high)
            summed_psdR2 = compute_summed_psd(normalized_psdsR2,freqsR2, low,high)
            
            centroid_band_R1 = compute_centroid(normalized_psdsR1, freqsR1, low, high)
            centroid_band_T = compute_centroid(normalized_psdsT, freqsT, low, high)
            centroid_band_R2 = compute_centroid(normalized_psdsR2, freqsR2, low, high)
            
            centroidy_band_R1 = compute_centroid_y(normalized_psdsR1, freqsR1, low, high)
            centroidy_band_T = compute_centroid_y(normalized_psdsT, freqsT, low, high)
            centroidy_band_R2 = compute_centroid_y(normalized_psdsR2, freqsR2, low, high)

            entropy_band_R1 = compute_entropy(normalized_psdsR1, low, high)
            entropy_band_T = compute_entropy(normalized_psdsT, low, high)
            entropy_band_R2 = compute_entropy(normalized_psdsR2, low, high)

            # Create a DataFrame for the centroids and entropies
            df_band = pd.DataFrame({
                'Band': band,
                'Label': label,
                'File': eeg_file,
                'CentroidR1': centroid_band_R1,
                'CentroidT': centroid_band_T,
                'CentroidR2': centroid_band_R2,
                'CentroidyR1': centroidy_band_R1,
                'CentroidyT': centroidy_band_T,
                'CentroidyR2': centroidy_band_R2,
                'EntropyR1': entropy_band_R1,
                'EntropyT': entropy_band_T,
                'EntropyR2': entropy_band_R2,
                'Summed PSDR1': summed_psdR1,
                'Summed PSDT' : summed_psdT,
                'Summed PSDR2': summed_psdR2
            })

            # Append the DataFrame to the list
            dfs.append(df_band)

                
#        for channel_idx in range(n_channels):
#            for band_name, (fmin, fmax) in freq_bands.items():
#                plt.figure(figsize=(10, 6))
#                band_idx_R1 = np.where((freqsR1 >= fmin) & (freqsR1 <= fmax))[0]
#                band_idx_T = np.where((freqsT >= fmin) & (freqsT <= fmax))[0]
#                band_idx_R2 = np.where((freqsR2 >= fmin) & (freqsR2 <= fmax))[0]
                
                # Calculate width of the figure based on frequency range proportion
#                width = 100 * ((fmax - fmin) / 45)  # base width multiplied by the proportion of band range to total range
                
#                plt.figure(figsize=(width, 6))
                
#                plt.plot(freqsR1[band_idx_R1], normalized_psdsR1[channel_idx, band_idx_R1], label=f'Rest Awal')
#                plt.plot(freqsT[band_idx_T], normalized_psdsT[channel_idx, band_idx_T], label=f'Tafakur')
#                plt.plot(freqsR2[band_idx_R2], normalized_psdsR2[channel_idx, band_idx_R2], label=f'Rest Akhir')
                
#                plt.xlabel('Frekuensi (Hz)', fontsize=20)
#                plt.ylabel('PSD Ternormalisasi', fontsize=20)
#                plt.legend(fontsize=12, loc='upper right')
#                plt.xticks(fontsize=18)
#                plt.yticks(fontsize=18)
#                plt.grid(True)
#                plt.tight_layout()
#                plt.figtext(0.5, 0.8, f'{channel_names[channel_idx]} - {band_name} - {short_label}', fontsize=24, ha='center')
#                plt.show()
                # Save the plot with a descriptive filename
                #plot_filename = f'{channel_names[channel_idx]}_{short_label}_{band_name}.png'
                #plot_filepath = os.path.join("D:\VSCode\Git Repositories\AlgorithmsNeuroscience\Tutorials\For Thesis\PSD", plot_filename)
                #plt.savefig(plot_filepath, bbox_inches="tight")
                
                #plt.close()  # Close the figure to avoid memory issues
                
                #print(f"Plot {channel_names[channel_idx]}_{band_name}_{short_label} saved successfully.")
                
                
#        # Calculate summed PSD for each condition
#        summed_psd_R1 = calculate_summed_psd(normalized_psdsR1, freqsR1, freq_bands)
#        summed_psd_T = calculate_summed_psd(normalized_psdsT, freqsT, freq_bands)
#        summed_psd_R2 = calculate_summed_psd(normalized_psdsR2, freqsR2, freq_bands)
        
        # Append the summed PSD values to the list
#        summed_psd_list.append({
#            'Label': label,
#            'File': eeg_file,
#            'Summed PSD Rest 1': summed_psd_R1,
#            'Summed PSD Tafakur': summed_psd_T,
#            'Summed PSD Rest 2': summed_psd_R2
#        })
# Create a DataFrame
#df = pd.DataFrame(summed_psd_list)
# Optionally, you can save the DataFrame to a CSV file
#df.to_csv('summed_psd_data.csv', index=False)



Reading D:\VSCode\Tesis\Pengukuran\Raw\Base\L3\A1_003_s_rmb.fdt
Reading 0 ... 124415  =      0.000 ...   971.992 secs...


C:\Users\hp\AppData\Local\Temp\ipykernel_10316\2395476139.py:6: RuntimeWarning: Data file name in EEG.data (003_s_rmb.fdt) is incorrect, the file name must have changed on disk, using the correct file name (A1_003_s_rmb.fdt).
  raw = mne.io.read_raw_eeglab(os.path.join(folder, eeg_file), preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 213 samples (1.664 s)



Fitting ICA to data using 5 channels (please be patient, this may take a while)
Selecting by number: 5 components
Fitting ICA took 1.2s.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\hp\AppData\Local\Temp\ipykernel_10316\1468158900.py:19: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  ic_labels = label_components(raw_filter, ica, method="iclabel")
C:\Users\hp\AppData\Local\Temp\ipykernel_10316\1468158900.py:19: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(raw_filter, ica, method="iclabel")
C:\Users\hp\AppData\Local\Temp\ipykernel_10316\1468158900.py:19: RuntimeWarning: The provided ICA instance was fitted with a 'fastica' algorithm. ICLabel was designed with extended infomax ICA decompositions. To use th

...

Applying ICA to Raw instance
    Transforming to ICA space (5 components)
    Zeroing out 1 ICA component
    Projecting back using 5 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 213 samples (1.664 s)

Fitting ICA to data using 5 channels (please be patient, this may take a while)
Selecting by number: 5 components
Fitting ICA took 1.3s.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\hp\AppData\Local\Temp\ipykernel_10316\1468158900.py:19: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  ic_labels = label_components(raw_filter, ica, method="iclabel")
C:\Users\hp\AppData\Local\Temp\ipykernel_10316\1468158900.py:19: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(raw_filter, ica, method="iclabel")
C:\Users\hp\AppData\Local\Temp\ipykernel_10316\1468158900.py:19: RuntimeWarning: The provided ICA instance was fitted with a 'fastica' algorithm. ICLabel was designed with extended infomax ICA decompositions. To use th

Applying ICA to Raw instance
    Transforming to ICA space (5 components)
    Zeroing out 1 ICA component
    Projecting back using 5 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 213 samples (1.664 s)

Fitting ICA to data using 5 channels (please be patient, this may take a while)
Selecting by number: 5 components
Fitting ICA took 0.1s.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\hp\AppData\Local\Temp\ipykernel_10316\1468158900.py:19: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  ic_labels = label_components(raw_filter, ica, method="iclabel")
C:\Users\hp\AppData\Local\Temp\ipykernel_10316\1468158900.py:19: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(raw_filter, ica, method="iclabel")
C:\Users\hp\AppData\Local\Temp\ipykernel_10316\1468158900.py:19: RuntimeWarning: The provided ICA instance was fitted with a 'fastica' algorithm. ICLabel was designed with extended infomax ICA decompositions. To use th

Applying ICA to Raw instance
    Transforming to ICA space (5 components)
    Zeroing out 2 ICA components
    Projecting back using 5 PCA components
Effective window size : 16.000 (s)
Effective window size : 16.000 (s)
Effective window size : 16.000 (s)


In [8]:

# Concatenate all data frames in dfs into one
merged_df = pd.concat(dfs, ignore_index=True)
#save in excel
excel_filename = "D:\VSCode\Git Repositories\AlgorithmsNeuroscience\Tutorials\For Thesis/centroid_data.xlsx"
with pd.ExcelWriter(excel_filename, engine = 'openpyxl', mode='a') as writer:
    merged_df.to_excel(writer, sheet_name='Sheet7', index=False)

print(f"New sheets added to Entropy")


New sheets added to Entropy


# Initialize lists to store results
centroid_results = []
entropy_results = []

# Loop through frequency bands
for band, (low, high) in freq_bands.items():
    # Calculate centroid and entropy for each frequency band
    centroid_band_R1 = compute_centroid(normalized_psdsR1, freqsR1, low, high)
    centroid_band_T = compute_centroid(normalized_psdsT, freqsT, low, high)
    centroid_band_R2 = compute_centroid(normalized_psdsR2, freqsR2, low, high)
    
    entropy_band_R1 = compute_entropy(normalized_psdsR1, low, high)
    entropy_band_T = compute_entropy(normalized_psdsT, low, high)
    entropy_band_R2 = compute_entropy(normalized_psdsR2, low, high)
    
    # Append results to lists
    centroid_results.append([centroid_band_R1, centroid_band_T, centroid_band_R2])
    entropy_results.append([entropy_band_R1, entropy_band_T, entropy_band_R2])

# Convert lists to numpy arrays for easier manipulation
centroid_array = np.array(centroid_results)
entropy_array = np.array(entropy_results)

# Reshape centroid and entropy arrays to match the shape of your PSD arrays
centroid_array = centroid_array.reshape((n_channels, -1))
entropy_array = entropy_array.reshape((n_channels, -1))

# Create pandas DataFrame to store results
df = pd.DataFrame(centroid_array, columns=[f'{band}_centroid_R1' for band in freq_bands] + 
                                          [f'{band}_centroid_T' for band in freq_bands] + 
                                          [f'{band}_centroid_R2' for band in freq_bands])

df_entropy = pd.DataFrame(entropy_array, columns=[f'{band}_entropy_R1' for band in freq_bands] + 
                                              [f'{band}_entropy_T' for band in freq_bands] + 
                                              [f'{band}_entropy_R2' for band in freq_bands])


# Combine centroid and entropy DataFrames
df_combined = pd.concat([df, df_entropy], axis=1)

# Add channel names as index
df_combined.index = channel_names

# Save DataFrame to Excel file
output_excel_path = 'centroid_entropy_results.xlsx'
df_combined.to_excel(output_excel_path)